In [1]:
import pandas as pd
import numpy as np
import os
import gc

import seaborn as sns
import matplotlib.pyplot as plt

import random

os.chdir("/home/elise/projects/whole-genome-tiling-predictions/")

In [10]:

print("reading files...")

N2a_human = pd.read_csv("scores/PB-PY-0011.hg38.scores.txt",
                       sep='\t')
print(N2a_human.head)


HepG2_human = pd.read_csv("scores/PB-PY-0012.hg38.scores.txt",
                       sep='\t')
print(HepG2_human.head)



reading files...
<bound method NDFrame.head of                                                    sequence condition  \
0         TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAAC...   Neuro2a   
1         ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC...   Neuro2a   
2         CCTAACCCAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA...   Neuro2a   
3         TAACCCTAACCCTAACCCTAACCCTAACCTAACCCTAACCCTAACC...   Neuro2a   
4         CCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCC...   Neuro2a   
...                                                     ...       ...   
64334862  TTTAGACCTTTAAGGTGAGTGTTACAGCACTGAAAGATGTTATGTC...   Neuro2a   
64334863  GTTTTTTCCTTCAGATATTTCCAGAGTTTCTTCCATCTCGCAGGTT...   Neuro2a   
64334864  GTCTTGGTCACTTCAAGAATGAAGCTGCAGACCTTAGTGGTGAGGG...   Neuro2a   
64334865  AGCACTTAAAGGTGTTATGTCCAGAGTTTTTTCCTACAGATGTGTC...   Neuro2a   
64334866  GTTTCTTCCTTCTGGCGGGTTCATGGTCTTGCTCACTTCAAGAATG...   Neuro2a   

          prediction                              region  
0          -0.007

KeyError: 'sequence'

In [ ]:


both_human = (N2a_human
                  .rename(columns={'prediction':'N2a'})
                  .filter(['region','sequence','N2a'])
                  .merge(HepG2_human
                         .rename(columns={'prediction':'HepG2'})
                         .filter(['region','sequence','HepG2']),
                                  on=('region','sequence'))
                  .assign(spec='human') )
#              .assign(diff=lambda x: x['HepG2']-x['N2a']) )


In [6]:
print(both_human.shape)
print(both_human.head)

#both_human['N2a'] = pd.to_numeric(HepG2_all['N2a'])
#both_human['HepG2'] = pd.to_numeric(HepG2_all['HepG2'])
#both_human['diff'] = both_human['HepG2'] - both_human['N2a']

#both_human.to_csv("scores/both.hg38.scores.diff_seqs_clust.txt",
#                                          sep='\t',
#                                         index=False)



NameError: name 'both_human' is not defined

In [7]:
del N2a_human
del HepG2_human
gc.collect()

In [ ]:
both_human_keep = pd.DataFrame(columns=both_human.columns)


for i in range(-10,11):
    for j in range(-10,11):
        both_human_keep.append( both_human
                                  .query("i < N2a <= i+1")
                                  .query("j < HepG2 <= j+1")
                                  .sample(500, replace=False) )



In [ ]:
sns.displot(both_human_keep,
           x='HepG2', y='N2a')

In [ ]:



N2a_mouse = pd.read_csv("scores/PB-PY-0011.mm10.scores.txt",
                       sep='\t')
print(N2a_mouse.head)


HepG2_mouse = pd.read_csv("scores/PB-PY-0012.mm10.scores.txt",
                       sep='\t')
print(HepG2_mouse.head)


both_human = (N2a_mouse
                  .rename(columns={'prediction':'N2a'})
                  .filter(['chr','pos','sequence','N2a'])
                  .merge(HepG2_human
                         .rename(columns={'prediction':'HepG2'})
                         .filter(['chr','pos','sequence','HepG2']),
                                  on=('chr','pos','sequence'))
              .assign(spec='human') )

In [ ]:
print(N2a_human.shape)
print(N2a_mouse.head)

In [ ]:
N2a_all = ( N2a_human
              .assign(model='N2a',spec='human')
                      .append( N2a_mouse
                                .assign(model='N2a',spec='mouse') ) )
N2a_all['prediction'] = pd.to_numeric(N2a_all['prediction'])

HepG2_all = ( HepG2_human
              .assign(model='HepG2',spec='human')
                      .append( HepG2_mouse
                                .assign(model='HepG2',spec='mouse') ) )
HepG2_all['prediction'] = pd.to_numeric(HepG2_all['prediction'])

#both_long = (N2a_all
#                .append(HepG2_all))
#                #.rename(columns={0:'pred_act',1:'chr',2:'pos'}))

both_merged = (N2a_all
                  .rename(columns={'prediction':'N2a'})
                  .filter(['N2a','chr','pos','spec'])
                  .merge(HepG2_all
                         .rename(columns={'prediction':'HepG2'})
                         .filter(['HepG2','chr','pos','spec',]),
                                  on=('chr','pos','spec'))
              .assign(diff=lambda x: x['HepG2']-x['N2a']) )

del N2a_all
del HepG2_all

gc.collect()

In [ ]:
both_merged = (both_merged
               .filter(['chr','pos','spec','HepG2','N2a','diff'])
               .drop_duplicates())

#rint(both_long.shape)
print(both_merged.shape)
print(both_merged.head)

print(both_merged.query("abs(diff) > 1"))
print(both_merged.query("abs(diff) > 1 & \
                            (HepG2<-.5 | HepG2>.5) & \
                            (N2a<-.5 | N2a>.5) & \
                            HepG2*N2a<1"))

print(both_merged.query("abs(diff) > 2"))
print(both_merged.query("abs(diff) > 2 & \
                            (HepG2<-.5 | HepG2>.5) & \
                            (N2a<-.5 | N2a>.5) & \
                            HepG2*N2a<1"))

print(both_merged.query("abs(diff) > 3"))
print(both_merged.query("abs(diff) > 3 & \
                            (HepG2<-.5 | HepG2>.5) & \
                            (N2a<-.5 | N2a>.5) & \
                            HepG2*N2a<1"))

In [8]:

both_merged.query("abs(diff) > 1").to_csv("scores/all.absdiff1.txt",
                                          sep='\t',
                                         index=False)

both_merged.query("abs(diff) > 2").to_csv("scores/all.absdiff2.txt",
                                          sep='\t',
                                         index=False)

both_merged.query("abs(diff) > 2 & \
                            (HepG2<-.5 | HepG2>.5) & \
                            (N2a<-.5 | N2a>.5) & \
                            HepG2*N2a<1").to_csv("scores/all.absdiff2_filt.txt",
                                          sep='\t',
                                         index=False)


In [ ]:
sns.set(style="whitegrid")
sns.scatterplot(data=both_merged,
                x='HepG2',
                y='N2a',
               s=10)
plt.plot([-6,6],[-6,6],
        linewidth=1,
        color='black')

sns.set(style="whitegrid")
g = sns.FacetGrid(
    data=both_merged,
    col='spec')
g.map(sns.scatterplot,
                'HepG2',
                'N2a',
               s=6)

def const_line(*args, **kwargs):
    x=np.arange(-6,6,.25)
    y=1*x
    plt.plot(y,x,
             linewidth=1,
        color='black')

g.map(const_line)



In [ ]:
sns.displot(data=both_merged_1M,
                x='HepG2',
                y='N2a',
           binwidth=(.25,.25))
plt.plot([-6,6],[-6,6],
        linewidth=1,
        color='black')

sns.displot(data=both_merged_1M,
                x='HepG2',
                y='N2a',
           hue="spec",
           binwidth=(.25,.25))
plt.plot([-6,6],[-6,6],
        linewidth=1,
        color='black')

sns.displot(data=both_merged_1M,
                x='HepG2',
                y='N2a',
           hue="spec",
           binwidth=(.25,.25),
           col='spec').map(const_line)
plt.plot([-6,6],[-6,6],
        linewidth=1,
        color='black')

NameError: name 'read' is not defined

In [14]:
#del N2a_human
#del N2a_mouse
del HepG2_human
del HepG2_mouse

NameError: name 'HepG2_human' is not defined

In [8]:
print(dir())
gc.collect()

['In', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_ih', '_ii', '_iii', '_oh', 'exit', 'gc', 'get_ipython', 'np', 'os', 'pd', 'plt', 'quit', 'sns']


195